In [268]:
import pandas as pd
import numpy as np 
import matplotlib as mpl
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore')

## Data Pre-Processing

In [269]:
df_inflows = pd.read_csv("..\Datasets\Original\TotalFDIInflows_2016_2020_ValueUS$_byCountry.csv")
df_outflows = pd.read_csv("..\Datasets\Original\TotalFDIOutflows_2016_2020_ValueUS$_byCountry.csv")


In [270]:
## Keeping only relevant cols
cols = ["Area Code (M49)", "Area", "Year", "Value", "Unit"]
df_inflows_trimmed = df_inflows[cols]
df_outflows_trimmed = df_outflows[cols]

df_inflows_trimmed = df_inflows_trimmed.rename(columns={"Area Code (M49)": "M49_Code", "Area": "Country_Name"})
df_outflows_trimmed = df_outflows_trimmed.rename(columns={"Area Code (M49)": "M49_Code", "Area": "Country_Name"})

print(df_inflows_trimmed.columns)
print(df_outflows_trimmed.columns)

Index(['M49_Code', 'Country_Name', 'Year', 'Value', 'Unit'], dtype='object')
Index(['M49_Code', 'Country_Name', 'Year', 'Value', 'Unit'], dtype='object')


In [271]:
## Dealing with NA/Empty Cells

# Method 1 => Removing all empty rows
def remove_NaN_rows(df_in: pd.DataFrame, val_col_name: str):
    df_out = df_in.loc[df_in[val_col_name].notna(), :]
    return df_out

# Method 2 => Replacing NaN with mean value
def replace_NaN_with_mean(df_in: pd.DataFrame, val_col_name: str):
    ...

df_inflows_trimmed = remove_NaN_rows(df_in=df_inflows_trimmed, val_col_name="Value")
df_outflows_trimmed = remove_NaN_rows(df_in=df_outflows_trimmed, val_col_name="Value")
# df1 = replace_NaN_with_mean(df_in=df1, val_col_name="Value")

## Data Processing

In [272]:
year_l = [2016, 2017, 2018, 2019, 2020]

In [273]:
## Transposing Value Column Data
"""
Steps:
    1) Generate dictionary: k == Year, v == list of values corresponding to the year
    2) Create 5 new cols for each Year
    3) Assign values into the 5 cols based on dictionary
    4) Drop unnecessary cols
    5) Shift values into a single row and remove extra rows
"""
def transpose_data(df1: pd.DataFrame, suffix: str):
    d = {}
    for y in year_l:
        d[y] = df1.loc[df1["Year"] == y, "Value"]

    for j in range(len(year_l)):
        col_name = f"Total_FDI_{suffix}_{year_l[j]}"
        df1[col_name] = 0

    for k,v in d.items():
        df1.loc[v.index, f"Total_FDI_{suffix}_{str(k)}"] = v

    df1 = df1.drop(["Year", "Value"], axis=1, inplace=False)

    return df1

df_inflows_transformed = transpose_data(df1=df_inflows_trimmed, suffix="Inflows")
df_outflows_transformed = transpose_data(df1=df_outflows_trimmed, suffix="Outflows")


In [274]:
## Condensing sparse matrix into a condensed form

def condense_df(df: pd.DataFrame, suffix: str):
    country_l = pd.DataFrame(df["Country_Name"].unique())
    m49_code_l = pd.DataFrame(df["M49_Code"].unique())

    df_out = pd.concat([m49_code_l, country_l], axis=1)
    df_out.columns = ["M49_Code", "Country_Name"]

    for i in year_l:
        fdi_df = pd.DataFrame()
        fdi_df = df.loc[df[f"Total_FDI_{suffix}_{i}"] != 0]
        fdi_df = fdi_df[["M49_Code", f"Total_FDI_{suffix}_{i}"]]
        df_out = pd.merge(df_out, fdi_df, on="M49_Code", how="outer")

    return df_out

inflow_fdi = condense_df(df_inflows_transformed, "Inflows")
outflow_fdi = condense_df(df_outflows_transformed, "Outflows")
outflow_fdi = outflow_fdi.drop(["Country_Name"], axis=1)


## Combining Inflows and Outflows FDI dfs into 1
net_fdi = inflow_fdi.merge(outflow_fdi, on="M49_Code", how="outer")[:-1]
# net_fdi = net_fdi.rename(columns={"Area Code (M49)": "M49_Code"})[:-1]
net_fdi

,M49_Code,Country_Name,Total_FDI_Inflows_2016,Total_FDI_Inflows_2017,Total_FDI_Inflows_2018,Total_FDI_Inflows_2019,Total_FDI_Inflows_2020,Total_FDI_Outflows_2016,Total_FDI_Outflows_2017,Total_FDI_Outflows_2018,Total_FDI_Outflows_2019,Total_FDI_Outflows_2020
0,4,Afghanistan,93.590000,53.390000,119.440000,38.533333,12.970492,14.780000,10.800000,40.530000,25.946667,37.112627
1,8,Albania,1100.671891,1148.891224,1289.690235,1287.978184,1106.560092,64.200483,26.197174,82.560790,127.855221,89.433996
2,12,Algeria,1636.299236,1232.341924,1466.084654,1381.811818,1125.414826,46.202384,-28.605242,845.268059,30.989854,15.518489
3,24,Angola,-179.517619,-7397.295409,-6456.076413,-4098.478748,-1866.468113,273.005000,1352.005057,5.714168,-2349.425835,90.514469
4,660,Anguilla,60.253704,53.534815,55.784444,125.088889,26.304077,-1.646667,-0.579630,-0.556296,-0.927531,-0.902742
...,...,...,...,...,...,...,...,...,...,...,...,...
195,862,Venezuela (Bolivarian Republic of),1068.000000,-68.000000,886.000000,934.000000,958.666667,1041.000000,2234.000000,661.000000,1076.000000,-78.666667
196,704,Viet Nam,12600.000000,14100.000000,15500.000000,16120.000000,15800.000000,1000.000000,480.000000,598.000000,465.000000,380.000000
197,887,Yemen,-561.000000,-269.850000,-282.098333,-370.982778,NaN,0.787000,5.601000,3.605667,3.331222,NaN
198,894,Zambia,662.900000,1107.500000,408.438492,547.967910,234.031750,176.700000,-72.000000,32.200000,696.213537,133.399156


In [275]:
## Finding the net FDI

numeric_cols = ['Total_FDI_Inflows_2016','Total_FDI_Inflows_2017', 'Total_FDI_Inflows_2018','Total_FDI_Inflows_2019', 'Total_FDI_Inflows_2020','Total_FDI_Outflows_2016', 'Total_FDI_Outflows_2017','Total_FDI_Outflows_2018', 'Total_FDI_Outflows_2019','Total_FDI_Outflows_2020']
net_fdi[numeric_cols] = net_fdi[numeric_cols].apply(pd.to_numeric, errors="ignore")

for y in year_l:
    val = net_fdi[f"Total_FDI_Inflows_{y}"] - net_fdi[f"Total_FDI_Outflows_{y}"]
    net_fdi[f"Net_FDI_{y}"] = val

net_fdi = net_fdi.iloc[:-1, :]
net_fdi


In [276]:
# net_fdi.to_csv("../Datasets\Processed/Net_FDI_Processed.csv")